---
# **Projeto de um sistema de recomendação de músicas baseado em conteúdo**
---

# Import de pacotes e dataset

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
a = pd.read_csv('/content/drive/MyDrive/Faculdade/UniRitter/BIG DATA/Sistemas de Recomendação/Projeto - Recomendação Musicas/Datasets/1/artists.csv')
t = pd.read_csv('/content/drive/MyDrive/Faculdade/UniRitter/BIG DATA/Sistemas de Recomendação/Projeto - Recomendação Musicas/Datasets/1/tracks.csv')

# Análise e tratamento dos datasets

In [3]:
a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1104349 entries, 0 to 1104348
Data columns (total 5 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   id          1104349 non-null  object 
 1   followers   1104336 non-null  float64
 2   genres      1104349 non-null  object 
 3   name        1104349 non-null  object 
 4   popularity  1104349 non-null  int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 42.1+ MB


In [4]:
# 1104349 registros em 5 colunas

a.shape

(1104349, 5)

In [5]:
for n in a.columns:
    print(n)

id
followers
genres
name
popularity


In [6]:
# Breve visualização do dataset Artists

a[:3]

,id,followers,genres,name,popularity
0,0DheY5irMjBUeLybbCUEZ2,0.0,[],Armid & Amir Zare Pashai feat. Sara Rouzbehani,0
1,0DlhY15l3wsrnlfGio2bjU,5.0,[],ปูนา ภาวิณี,0
2,0DmRESX2JknGPQyO15yxg7,0.0,[],Sadaa,0


In [7]:
t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 586672 entries, 0 to 586671
Data columns (total 20 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                586672 non-null  object 
 1   name              586601 non-null  object 
 2   popularity        586672 non-null  int64  
 3   duration_ms       586672 non-null  int64  
 4   explicit          586672 non-null  int64  
 5   artists           586672 non-null  object 
 6   id_artists        586672 non-null  object 
 7   release_date      586672 non-null  object 
 8   danceability      586672 non-null  float64
 9   energy            586672 non-null  float64
 10  key               586672 non-null  int64  
 11  loudness          586672 non-null  float64
 12  mode              586672 non-null  int64  
 13  speechiness       586672 non-null  float64
 14  acousticness      586672 non-null  float64
 15  instrumentalness  586672 non-null  float64
 16  liveness          58

In [8]:
# 586672 registros em 20 colunas

t.shape

(586672, 20)

In [9]:
for n in t.columns:
    print(n)

id
name
popularity
duration_ms
explicit
artists
id_artists
release_date
danceability
energy
key
loudness
mode
speechiness
acousticness
instrumentalness
liveness
valence
tempo
time_signature


In [10]:
# Breve visualização do dataset Tracks

t[:3]

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.445,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.263,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.177,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5


**Data Cleaning:** 

Checando valores duplicados no dataset Artistas

In [11]:
a.duplicated().sum()

0

**Data Cleaning:**

Checando valores duplicados no dataset Tracks

In [12]:
t.duplicated().sum()

0

**Data Transformation:**

Começando o tratamento dos datasets com o intuito de transformar ambos em um único dataset para o cálculo de distância dos cosenos.

In [13]:
# Renomeando colunas

a.rename(columns = {'id':'id_artists', 'name':'artists', 'popularity':'artists_popularity'}, inplace = True)
t.rename(columns = {'id':'id_track', 'name':'track_title'}, inplace = True)

In [14]:
a.columns

Index(['id_artists', 'followers', 'genres', 'artists', 'artists_popularity'], dtype='object')

In [15]:
t.columns

Index(['id_track', 'track_title', 'popularity', 'duration_ms', 'explicit',
       'artists', 'id_artists', 'release_date', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature'],
      dtype='object')

In [16]:
# Combinando os datasets em um único

df = pd.merge(a, t, on=["id_artists", "artists"], how="right", validate='one_to_many')

In [17]:
# Parametrizando as colunas do novo dataset

columns = columns = ['artists', 'track_title', 'popularity', 'duration_ms', 'explicit',
       'release_date', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'time_signature']

df = pd.DataFrame(df, columns=columns)

In [18]:
# Realizando tratamento de strings na coluna artists, removendo caracteres desnecessários

df['artists'] = df['artists'].str.replace("[","").str.replace("'", "").str.replace("]","")

<ipython-input-18-1760cc729fbb>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['artists'] = df['artists'].str.replace("[","").str.replace("'", "").str.replace("]","")


In [19]:
# Verificação do tratamento

df.head(3)

,artists,track_title,popularity,duration_ms,explicit,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,Uli,Carve,6,126903,0,1922-02-22,0.645,0.445,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3
1,Fernando Pessoa,Capítulo 2.16 - Banquero Anarquista,0,98200,0,1922-06-01,0.695,0.263,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1
2,Ignacio Corsini,Vivo para Quererte - Remasterizado,0,181640,0,1922-03-21,0.434,0.177,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5


---
**Levando em consideração as informações contidas no *dataset* e o que se tem em mente (construir um sistema de recomendação de músicas). A abordagem inicial seria baseada em conteúdo, visto que não temos informações de usuários, somente informações brutas de seguidores e popularidade. O que nos permitiria utilizar um algorítimo de clusterização para construção das relações entre gênero musical, seguidores e popularidade (vinculadas aos artistas) para uma recomendação de artistas no caso.** 

**Ainda em relação ao dataset, temos diversas informações pertinentes para parametrizar uma recomendação baseada em conteúdo, sendo elas: 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence' e 'tempo'.**

**Sendo que ainda é possível se fazer um refinamento dessas colunas para tornar o sistema um pouco mais enxuto.**

---

# Salvando o novo dataset

In [20]:
# Salvando o novo dataset no Google Drive

from pathlib import Path 
filepath = Path('/content/drive/MyDrive/Faculdade/UniRitter/BIG DATA/Sistemas de Recomendação/Projeto - Recomendação Musicas/Datasets/1/artist_tracks.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath)  

# Análise Críticas
Em relação à abordagem colaborativa, os datasets não apresentam informações de usuários propriamente ditas, somente informações brutas de seguidores e popularidade. Não faria sentido, então levar em consideração essa abordagem.

Tendo isso em mente, a abordagem mais indicada seria a baseada em conteúdo, pois temos informação suficiente para dar seguimento por esse caminho. Podemos relacionar as informações relativas às faixas para gerar uma recomendação baseada nos seus atributos, bem como sua popularidade e até mesmo a data de lançamento.

Em relação à abordagem híbrida, seria algo um pouco mais complexo e talvez de baixo retorno em função do trabalho envolvido (retorno no sentido de precisão da recomendação) pois não temos informação suficiente para embasar uma recomendação colaborativa.

# Construção do sistema de recomendação

In [21]:
# Construção da função de recomendação

def recomendar_por_musica_similar(df, musica, artista):

    """
    Colunas originais do dataset utilizadas no sistema de recomendação.
    ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]
    """

    try: 
        musica_artista = df[(df['track_title'] == musica) & (df['artists'] == artista)]

        musica_similar = df.copy()

        propriedades = musica_similar.loc[:,['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness']]
 
        musica_similar['similaridade'] = cosine_similarity(propriedades, propriedades.to_numpy()[musica_artista.index[0],None]).squeeze()

        musica_similar.rename(columns = {'artists': 'Artistas', 
                                         'track_title': f'Músicas similares a {musica}',
                                         'release_date': 'Data de Lançamento',
                                         'popularity': 'Popularidade'
                                         }, inplace=True)

        musica_similar = musica_similar.sort_values(by= 'similaridade', ascending=False)

        musica_similar = musica_similar[['Artistas', 
                                         f'Músicas similares a {musica}',
                                         'Data de Lançamento', 
                                         'Popularidade']]
        
        musica_similar.reset_index(drop=True, inplace=True)

        return musica_similar.iloc[1:11]

    except:
        print('Oh noes!\nMúsica não localizada na base de dados!')

# Teste da função de recomendação

In [22]:
recomendar_por_musica_similar(df, 'Outshined', 'Soundgarden')

,Artistas,Músicas similares a Outshined,Data de Lançamento,Popularidade
1,Adelitas Way,Last Stand,2009-01-01,57
2,"Jumper, Olsson, Pontus & Sundquist",Hon har ett sätt,1997,24
3,Ted Nugent,Dog Eat Dog,1976,36
4,deLillos,Neste sommer,2002-01-01,25
5,Midnight Oil,Dust - Remastered Version,1978,16
6,Sestrumer,Aria D'inverno - Studio Version 2009,2019-12-02,0
7,"DJ Sammy, Do",Heaven,2013-08-12,54
8,La Beriso,Tan Sola,2009-12-18,41
9,Blind Guardian,Nightfall - Remastered 2007,1998-04-28,49
10,Strachy Na Lachy,Żyję w kraju,2010,44


In [23]:
recomendar_por_musica_similar(df, 'One More Time', 'Daft Punk')

,Artistas,Músicas similares a One More Time,Data de Lançamento,Popularidade
1,Ariana Grande,Santa Tell Me,2014-11-24,68
2,Los Abuelos De La Nada,No Se Desesperen,1995-01-01,35
3,Catupecu Machu,Calavera Deforme,1997-01-26,28
4,Sublime,Date Rape,1996-07-30,33
5,The Police,On Any Other Day,1979-10-02,44
6,Udit Narayan,Dil Deewana Kehta Hai,1999-05-07,28
7,Kindan no Tasuketsu,BABY,2018-10-19,3
8,Megas,(Borðið þér) Orma frú Norma,1988-01-01,2
9,Sektor Gaza,Свин,1997,19
10,Libro,胎動,1998,28


In [24]:
recomendar_por_musica_similar(df, 'Basket Case', 'Green Day')

,Artistas,Músicas similares a Basket Case,Data de Lançamento,Popularidade
1,Hoodoo Gurus,Middle Of The Land - 2005 Remaster,1987-07-17,20
2,Gackt,Journey through the Decade,2009-03-25,42
3,Rhapsody,Triumph for My Magic Steel,2000,44
4,Sum 41,Walking Disaster,2008-01-01,6
5,The Cousins,Kili Watch,1960,19
6,MEGA NRG MAN,BACK ON THE ROCKS,1998-07-29,52
7,Bad Religion,Against the Grain,1990,38
8,La Renga,El Ojo del Huracán,2003-01-13,43
9,Kraftklub,Alles wegen Dir,2014-01-01,47
10,Fall Out Boy,Centuries,2015-01-20,82


In [25]:
recomendar_por_musica_similar(df, 'Enjoy the Silence', 'Depeche Mode')

,Artistas,Músicas similares a Enjoy the Silence,Data de Lançamento,Popularidade
1,Depeche Mode,Enjoy the Silence - 2006 Remaster,1990-03-19,55
2,"Efraim Shamir, Astar Shamir",המלך ערום,1977-01-01,6
3,Dion,Lonely Teenager,1961-01-01,4
4,Bob Dylan,Tangled up in Blue,1975-01-17,65
5,Dinah Washington,When Your Lover Has Gone,1962-01-01,2
6,The Paul Winter Sextet,The Spell of the Samba (Samba da Minha Terra),1962-08-26,18
7,Eiichi Ohtaki,Water Color (B-EACH TIME L-ONG Version),1985-06-01,34
8,Mika Karni,כל קיץ,1997-01-01,23
9,Lynyrd Skynyrd,Comin' Home,1991-11-12,36
10,Runrig,Loch Lomond,1979-08-01,25


In [26]:
recomendar_por_musica_similar(df, "Friday I'm In Love", "The Cure")

,Artistas,Músicas similares a Friday I'm In Love,Data de Lançamento,Popularidade
1,The Cure,Friday I'm in Love,1992-04-21,68
2,Extremoduro,La hoguera,1989,31
3,The Angels,Dogs Are Talking,1990-07-17,42
4,Pescado Rabioso,Blues de Cris,1976-11-28,29
5,Dewa 19,Hitam Putih,1995,34
6,Dio,Eat Your Heart Out,1984,32
7,Camisa de Vênus,Meu Primo Zé,1983-01-01,21
8,Karel Gott,Die Biene Maja,1994-01-01,44
9,Poison,Talk Dirty To Me,1986-01-01,54
10,De Press,Bo Jo Cie Kochom,1981-01-01,26


In [27]:
recomendar_por_musica_similar(df, "Twilight Of The Thunder God", "Amon Amarth")

,Artistas,Músicas similares a Twilight Of The Thunder God,Data de Lançamento,Popularidade
1,Falcata,Dark Times,2020-06-25,1
2,川田まみ,緋色の空,2006-03-29,41
3,Oag,60's TV,2000-01-01,20
4,Anne-Lie Rydé,Alla har glömt,1992-01-01,18
5,Arctic Monkeys,Leave Before The Lights Come On,2006-08-14,58
6,Masatoshi Ono,Departure!,2013-09-18,69
7,Gorilla Biscuits,Cats and Dogs,1989-07-14,30
8,Midnight Oil,Written In The Heart - Remastered Version,1981,19
9,BUCK-TICK,...IN HEAVEN...,1992-03-21,29
10,Thee Michelle Gun Elephant,ゲット・アップ・ルーシー,1997-11-01,37


In [28]:
recomendar_por_musica_similar(df, "New Day", "Karnivool")

,Artistas,Músicas similares a New Day,Data de Lançamento,Popularidade
1,Marilyn Manson,Angel With The Scabbed Wings,1996,51
2,Matsuko Mawatari,Homework ga Owaranai,1992,25
3,Julio Iglesias,"Ae, Ao",1978,26
4,Yothu Yindi,Treaty - Radio Mix,1991-11-02,49
5,Platero Y Tu,Alucinante,1997,40
6,TM NETWORK,Don't Let Me Cry (一千一秒物語),1987,13
7,Tavares,Heaven Must Be Missing An Angel - Pt. 1,1993-06-21,51
8,Blakdyak,Hayop Na Combo,1997-11-06,32
9,Elton John,Club At The End Of The Street,1989-08-29,40
10,Mamonas Assassinas,Robocop Gay,1995-01-01,57


In [29]:
recomendar_por_musica_similar(df, "The Kids Aren't Alright", "The Offspring")

,Artistas,Músicas similares a The Kids Aren't Alright,Data de Lançamento,Popularidade
1,.ex-machina.,Pigs (2016 Version),2016-06-30,0
2,"Friction, Flowidus, Raphaella",By Your Side,2020-09-11,55
3,Oasis,Rockin' Chair,1998-11-03,43
4,Oasis,Rockin' Chair - Remastered,1995-10-02,38
5,HOTEI,SURRENDER,1994,32
6,Yutaka Ozaki,禁猟区,1984,22
7,Indio Solari y los Fundamentalistas del Aire A...,Black Russian,2010,42
8,T.Love,Stany,1992-10-11,16
9,Rammstein,BENZIN,2005-10-27,60
10,Rammstein,BENZIN,2005-10-27,49


In [30]:
recomendar_por_musica_similar(df, "Like a Stone", "Audioslave")

,Artistas,Músicas similares a Like a Stone,Data de Lançamento,Popularidade
1,Audioslave,Like a Stone,2002-11-17,52
2,Audioslave,Like a Stone,2002,46
3,Paradox,เขียนไว้ข้างเตียง,2019-11-14,2
4,Paradox,เขียนไว้ข้างเตียง,2018-11-14,41
5,Kotak,Jagalah Bumi - Theme from BoBoiBoy,2015-04-14,39
6,Girafot,קשה לי,2006-06-19,19
7,Prāta Vētra,Lec,2000-01-01,30
8,Julia Wieniawa,Nie muszę,2018-11-09,55
9,Tazenda,Pitzinnos in sa gherra,2007-06-22,30
10,Cartoons,Who Put the Bomp,1998-09-24,29


In [31]:
recomendar_por_musica_similar(df, "Blinding Lights", "The Weeknd")

,Artistas,Músicas similares a Blinding Lights,Data de Lançamento,Popularidade
1,Faye Wong,精彩,1999-10-14,22
2,Shocking Blue,Gonna Sing Me A Song,1975-09-06,23
3,Cody Jinks,Mamma Song,2015-01-12,64
4,Incubus,Pardon Me,1999-10-26,69
5,Kárpátia,Szél viszi messze,2005,26
6,Matamba,Con Vos,2013-04-07,27
7,Lil Peep,Honestly,2017-06-08,65
8,Böhse Onkelz,Kirche,1996-09-02,54
9,La Tabaré,Excepto a Veces,2006-01-01,19
10,Eddie Meduza,Disco Burp - Special Version,1979,17


# Pesquisa de **artistas** e **músicas** no **dataset**
Ordenada por popularidade das músicas

In [39]:
# Basta substituir o nome do artista após o sinal de '==' 
# e o valor dentro dos colchetes [:xx], para o total de músicas apresentadas do artista

df[(df['artists'] == 'Rihanna')].sort_values(by='popularity', ascending=False)[:10]

,artists,track_title,popularity,duration_ms,explicit,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
88970,Rihanna,Needed Me,81,191600,1,2016-01-28,0.671,0.314,5,-8.091,0,0.2440,0.110000,0.000000,0.0825,0.296,110.898,4
88964,Rihanna,Love On The Brain,81,224000,1,2016-01-28,0.509,0.637,4,-4.830,0,0.0471,0.071700,0.000011,0.0789,0.378,172.006,3
78366,Rihanna,Pon de Replay,80,246960,0,2005-08-29,0.779,0.640,7,-8.415,1,0.1590,0.000155,0.000770,0.1010,0.498,99.019,4
81267,Rihanna,Don't Stop The Music,79,267080,0,2008-06-02,0.835,0.669,6,-5.582,0,0.0643,0.033600,0.000069,0.0535,0.542,122.668,4
89037,Rihanna,Desperado,78,186467,0,2016-01-28,0.685,0.610,0,-5.221,0,0.0439,0.323000,0.000000,0.1020,0.323,94.384,3
81275,Rihanna,Disturbia,78,238627,0,2008-06-02,0.707,0.813,11,-4.515,0,0.0571,0.086300,0.000000,0.1680,0.722,124.921,4
81289,Rihanna,Take A Bow,77,229413,0,2008-06-02,0.697,0.467,9,-7.536,1,0.0715,0.248000,0.000000,0.0941,0.572,82.082,4
88041,Rihanna,Bitch Better Have My Money,77,219306,1,2015-03-26,0.781,0.728,1,-4.981,1,0.0621,0.050900,0.000002,0.2570,0.395,102.990,4
85096,Rihanna,Diamonds,77,225147,0,2012-12-11,0.564,0.710,11,-4.920,0,0.0461,0.001250,0.000000,0.1090,0.393,91.972,4
82194,Rihanna,Rude Boy,76,222920,1,2009-11-20,0.563,0.750,11,-4.496,1,0.1270,0.113000,0.000000,0.0788,0.812,173.906,4


# Recomendação

basta digitar o nome da função passando como parâmetro o **dataset**, o **nome da música** e o **artista** como no exemplo:                   

```
#                           Dataset,   Música,     Artista   
recomendar_por_musica_similar(df, 'Outshined', 'Soundgarden')
```

In [43]:
recomendar_por_musica_similar(df, "Treasure", "Bruno Mars")

,Artistas,Músicas similares a Before I Forget,Data de Lançamento,Popularidade
1,"""Gasolin""",Refrainet Er Frit - Live,1976-08-30,9
2,COMPLEX,IMAGINE HEROES,1989-04-26,22
3,Ossian,A rock katonái,2011-12-05,31
4,Sonata Arctica,The Wolves Die Young - (Live at Wacken 2017),2018-07-20,4
5,COMPLEX,PRETTY DOLL - Live,1991-01-23,19
6,Scissor Sisters,"She's My Man - Live At The O2, London, UK / 2007",2007-01-01,0
7,Turbonegro,Rock Against Ass,1998-01-01,30
8,Miel San Marcos,Grande Y Fuerte,2012-11-10,50
9,Queens of the Stone Age,Little Sister - Live At Brixton Academy / 2005,2005-11-22,27
10,Klamydia,Niilo Ahkio,2009-02-25,29
